In [4]:
import argparse
import sys
import os
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
import matplotlib.pyplot as plt
from models import Generator
from datasets import ImageDataset

parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=1, help='size of the batches')
parser.add_argument('--dataroot', type=str, default='datasets/duke2market/', help='root directory of the dataset')
parser.add_argument('--input_nc', type=int, default=3, help='number of channels of input data')
parser.add_argument('--output_nc', type=int, default=3, help='number of channels of output data')
parser.add_argument('--size', type=int, default=256, help='size of the data (squared assumed)')
parser.add_argument('--cuda', action='store_true', help='use GPU computation')
parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
parser.add_argument('--generator_A2B', type=str, default='output/7/netG_A2B.pth', help='A2B generator checkpoint file')
parser.add_argument('--generator_B2A', type=str, default='output/7/netG_B2A.pth', help='B2A generator checkpoint file')
opt = parser.parse_args(['--cuda'])

In [5]:
# Function to convert tensor to image for displaying
def tensor_to_image(tensor):
    tensor = tensor.clone().detach().cpu()  # Clone the tensor to avoid modifying the original and move to CPU
    tensor = tensor.squeeze(0)  # Remove batch dimension
    tensor = tensor * torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1) + torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1)  # De-normalize
    tensor = tensor.permute(1, 2, 0)  # Change from CxHxW to HxWxC
    tensor = tensor.numpy()  # Convert to numpy array
    tensor = (tensor * 255).astype(np.uint8)  # Convert to uint8
    return tensor

# Function to deprocess and display image
def deprocess_image(image):
    image = image.squeeze(0)  # Remove batch dimension
    image = image * 0.5 + 0.5  # De-normalize to [0, 1]
    image = image.clamp(0, 1)
    return transforms.ToPILImage()(image)

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

###### Definition of variables ######
# Networks
G_duke_market = Generator(opt.input_nc, opt.output_nc)
G_market_duke = Generator(opt.output_nc, opt.input_nc)

if opt.cuda:
    G_duke_market.cuda()
    G_market_duke.cuda()

# Load state dicts
G_duke_market.load_state_dict(torch.load(opt.generator_A2B))
G_market_duke.load_state_dict(torch.load(opt.generator_B2A))

# Set model's test mode
G_duke_market.eval()
G_market_duke.eval()

# Inputs & targets memory allocation
Tensor = torch.cuda.FloatTensor if opt.cuda else torch.Tensor
input_A = Tensor(opt.batchSize, opt.input_nc, opt.size, opt.size)
input_B = Tensor(opt.batchSize, opt.output_nc, opt.size, opt.size)

crop_size_w = 128
crop_size_h = 256
# Dataset loader
transform = transforms.Compose([
    #transforms.Resize((crop_size_h, crop_size_w)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [1]:
# Import library
from pathlib import Path

# Market1501 dataset dir
raw_data_dir_market1501 = Path('/home/jun/ReID_Dataset/Market-1501-v15.09.15')
# Make dictionary of all image market1501 with keys of image_name and values of image_path
market_train_dic = {i.name:i for i in sorted(list(raw_data_dir_market1501.glob('bounding_box_train/*.jpg')))}
market_gallery_dic = {i.name:i for i in sorted(list(raw_data_dir_market1501.glob('bounding_box_test/*.jpg')))}
market_query_dic = {i.name:i for i in sorted(list(raw_data_dir_market1501.glob('query/*.jpg')))}

import random

# Assuming market_train_dic is already defined and populated with values
image_paths_list = list(market_train_dic.values())

# Shuffle the list to ensure randomness
random.shuffle(image_paths_list)

# Select 10 random paths
random_image_paths = image_paths_list[:1]
image_paths = list(market_train_dic.values())[200:201] 
#image_paths = random_image_paths

images = list(transform(Image.open(image_path).convert('RGB')) for image_path in image_paths)  # Ensure image is in RGB mode
# Apply the transformations
#transformed_image = transforms(image)
for image in images: 

    G_market_duke.eval()
    G_duke_market.eval()
    x = image.cuda()

    with torch.no_grad():
        y1 = G_market_duke(x)
        y2 = G_duke_market(x)
    # Convert tensors to images
    original_image_np = tensor_to_image(x.squeeze(0))  # Remove batch dimension for original image
    output_image_np1 = tensor_to_image(y1)
    output_image_np2 = tensor_to_image(y2)

    # Display the images
    plt.figure(figsize=(10, 5))

    # Display the original image
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(original_image_np)
    plt.axis('off')  # Hide the axes

    # Display the output image
    plt.subplot(1, 3, 2)
    plt.title("Style Transfered Image")
    plt.imshow(output_image_np1)
    plt.axis('off')  # Hide the axes

    # Display the output image
    plt.subplot(1, 3, 3)
    plt.title("Generated Original Image")
    plt.imshow(output_image_np2)
    plt.axis('off')  # Hide the axes
    plt.show()

# duke1501 dataset dir
raw_data_dir_duke = Path('/home/jun/ReID_Dataset/DukeMTMC-reID')
# Make dictionary of all image duke1501 with keys of image_name and values of image_path
duke_train_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('bounding_box_train/*.jpg')))}
duke_gallery_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('bounding_box_test/*.jpg')))}
duke_query_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('query/*.jpg')))}

import random

# Assuming market_train_dic is already defined and populated with values
image_paths_list = list(duke_train_dic.values())

# Shuffle the list to ensure randomness
random.shuffle(image_paths_list)

# Select 10 random paths
random_image_paths = image_paths_list[:1]
image_paths = list(duke_train_dic.values())[200:201] 
#image_paths = random_image_paths

images = list(transform(Image.open(image_path).convert('RGB')) for image_path in image_paths)  # Ensure image is in RGB mode
# Apply the transformations
#transformed_image = transforms(image)
for image in images: 

    G_market_duke.eval()
    G_duke_market.eval()
    x = image.cuda()

    with torch.no_grad():
        y2 = G_market_duke(x)
        y1 = G_duke_market(x)
    # Convert tensors to images
    original_image_np = tensor_to_image(x.squeeze(0))  # Remove batch dimension for original image
    output_image_np1 = tensor_to_image(y1)
    output_image_np2 = tensor_to_image(y2)

    # Display the images
    plt.figure(figsize=(10, 5))

    # Display the original image
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(original_image_np)
    plt.axis('off')  # Hide the axes

    # Display the output image
    plt.subplot(1, 3, 2)
    plt.title("Style Transfered Image")
    plt.imshow(output_image_np1)
    plt.axis('off')  # Hide the axes

    # Display the output image
    plt.subplot(1, 3, 3)
    plt.title("Generated Original Image")
    plt.imshow(output_image_np2)
    plt.axis('off')  # Hide the axes
    plt.show()

NameError: name 'transform' is not defined

In [6]:
new_data_dir_market1501 = Path('/home/jun/ReID_Dataset/Market-1501-v15.09.15-dukestyle-cycleGAN')
new_data_dir_market1501.mkdir(exist_ok=True)
new_market_train_dir = new_data_dir_market1501 / 'bounding_box_train'
new_market_train_dir.mkdir(exist_ok=True)
new_market_gallery_dir = new_data_dir_market1501 / 'bounding_box_test'
new_market_gallery_dir.mkdir(exist_ok=True)
new_market_query_dir = new_data_dir_market1501 / 'query'
new_market_query_dir.mkdir(exist_ok=True)


for image_name, image_path in market_train_dic.items():
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    # Apply the transformations
    transformed_image = transform(image)
    G_market_duke.eval()
    x = transformed_image.cuda()
    with torch.no_grad():
        y = G_market_duke(x)
    output_image = tensor_to_image(y)
    output_image = Image.fromarray(output_image)
    output_image_path = new_market_train_dir / image_name
    output_image.save(output_image_path)

for image_name, image_path in market_gallery_dic.items():
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    # Apply the transformations
    transformed_image = transform(image)
    G_market_duke.eval()
    x = transformed_image.cuda()
    with torch.no_grad():
        y = G_market_duke(x)
    output_image = tensor_to_image(y)
    output_image = Image.fromarray(output_image)
    output_image_path = new_market_gallery_dir / image_name
    output_image.save(output_image_path)
    
for image_name, image_path in market_query_dic.items():
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    # Apply the transformations
    transformed_image = transform(image)
    G_market_duke.eval()
    x = transformed_image.cuda()
    with torch.no_grad():
        y = G_market_duke(x)
    output_image = tensor_to_image(y)
    output_image = Image.fromarray(output_image)
    output_image_path = new_market_query_dir / image_name
    output_image.save(output_image_path)

In [7]:
# Import library
from pathlib import Path

# duke1501 dataset dir
raw_data_dir_duke = Path('/home/jun/ReID_Dataset/DukeMTMC-reID')
# Make dictionary of all image duke1501 with keys of image_name and values of image_path
duke_train_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('bounding_box_train/*.jpg')))}
duke_gallery_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('bounding_box_test/*.jpg')))}
duke_query_dic = {i.name:i for i in sorted(list(raw_data_dir_duke.glob('query/*.jpg')))}

new_data_dir_duke = Path('/home/jun/ReID_Dataset/DukeMTMC-reID-marketstyle-cycleGAN')
new_data_dir_duke.mkdir(exist_ok=True)
new_duke_train_dir = new_data_dir_duke / 'bounding_box_train'
new_duke_train_dir.mkdir(exist_ok=True)
new_duke_gallery_dir = new_data_dir_duke / 'bounding_box_test'
new_duke_gallery_dir.mkdir(exist_ok=True)
new_duke_query_dir = new_data_dir_duke / 'query'
new_duke_query_dir.mkdir(exist_ok=True)

In [8]:
for image_name, image_path in duke_train_dic.items():
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    # Apply the transformations
    transformed_image = transform(image)
    G_duke_market.eval()
    x = transformed_image.cuda()
    with torch.no_grad():
        y = G_duke_market(x)
    output_image = tensor_to_image(y)
    output_image = Image.fromarray(output_image)
    output_image_path = new_duke_train_dir / image_name
    output_image.save(output_image_path)

for image_name, image_path in duke_gallery_dic.items():
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    # Apply the transformations
    transformed_image = transform(image)
    G_duke_market.eval()
    x = transformed_image.cuda()
    with torch.no_grad():
        y = G_duke_market(x)
    output_image = tensor_to_image(y)
    output_image = Image.fromarray(output_image)
    output_image_path = new_duke_gallery_dir / image_name
    output_image.save(output_image_path)
    
for image_name, image_path in duke_query_dic.items():
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    # Apply the transformations
    transformed_image = transform(image)
    G_duke_market.eval()
    x = transformed_image.cuda()
    with torch.no_grad():
        y = G_duke_market(x)
    output_image = tensor_to_image(y)
    output_image = Image.fromarray(output_image)
    output_image_path = new_duke_query_dir / image_name
    output_image.save(output_image_path)

/home/jun/miniconda3/envs/py312/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608847532/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


: 